### DNN기반 회귀 모델 구현 + 학습 진행 모니터링
- 데이터셋  : iris.csv
- 피쳐/속성 : 3개 (Sepal_Length, Sepal_Width, Petal_Length)
- 타겟/라벨 : 1개 (Petal_Width)
- 학습 방법 : 지도학습 > 회귀
- 알고리즘  : 인공신경망(ANN) -> MLP, DNN : 은닉층이 많은 구성
- 프레임워크 : Pytorch
- - -
- 모니터링
    * 기준 : 검증 데이터셋의 loss 또는 score
    * 평가 : 학습 데이터셋의 loss 또는 score와 비교해서 학습 중단 여부 결정
    * 선택 : 현재까지 진행된 모델의 파라미터(가중치, 절편) 저장 여부 또는 모델 전체 저장

[1] 모듈 로딩 및 데이터 준비 <hr>

In [30]:
# 모듈 로딩
# Model 관련
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from torchmetrics.regression import R2Score, MeanSquaredError
from torchinfo import summary

# Data 및 시각화 관련
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import *
from sklearn.model_selection import train_test_split

In [31]:
# 활용 패키지 버전 체크 ==> 사용자 정의함수로 구현하세요~
print(f'Pytorch v.{torch.__version__}')
print(f'pandas v.{pd.__version__}')

Pytorch v.2.4.1
pandas v.2.0.3


In [32]:
# 데이터 로딩
DATA_FILE = '../../../Data/iris.csv'

# CSV >>> DataFrame
irisDF = pd.read_csv(DATA_FILE, usecols=[0,1,2,3])

# 확인
irisDF.head(2)

,sepal.length,sepal.width,petal.length,petal.width
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2


[2] 모델 클래스 설계 및 정의 <hr>
- 클래스 목적 : iris 데이터를 학습 및 추론 목적
- 클래스 이름 : IrisRegModel
- 부모 클래스 : nn.Module
- 매개   변수 : 층별 입출력 개수 고정하기 때문에 필요 없음
- 속성 / 필드 :
- 기능 / 역할 : \__init__() : 모델 구조 설정, forward() : 순방향 학습 <= 오버라이딩(overriding)

- 클래스 구조
    * 입력층 :
        * 입력 3개(피쳐)
        * 출력 10개(퍼셉트론/뉴런 10개 존재)
    * 은닉층 :
        * 입력 10개
        * 출력 30개(퍼셉트론/뉴런 30개 존재)
    * 출력층 :
        * 입력 30개
        * 출력 1개(너비)

- 손실함수/활성화함수
    * 클래스 형태 ==> nn.MSELoss, nn.ReLU ==> init() 메서드
    * 함수 형태 ==> torch.nn.functional 아래에 ==> forward() 메서드

In [33]:
class IrisRegModel(nn.Module):

    # 모델 구조 구성 및 인스턴스 생성 메서드
    def __init__(self):
        super().__init__()

        self.in_layer = nn.Linear(3, 10)
        self.hidden_layer = nn.Linear(10, 30)
        self.out_layer = nn.Linear(30, 1)

    # 순방향 학습 진행 메서드
    def forward(self, input_data):
        # 입력층
        y = self.in_layer(input_data)   # f1w1+f2w2+f3w3+b
        y = F.relu(y)                   # relu => y 값의 범위 : 0 <= y

        # 은닉층 : 10개의 숫자 값(>=0)
        y = self.hidden_layer(y)
        y = F.relu(y)

        # 출력층 : 30개의 숫자 값(>=0) 회귀이므로 바로 반환(return)
        return self.out_layer(y)

In [34]:
# 모델 인스턴스 생성
model = IrisRegModel()

print(model)

IrisRegModel(
  (in_layer): Linear(in_features=3, out_features=10, bias=True)
  (hidden_layer): Linear(in_features=10, out_features=30, bias=True)
  (out_layer): Linear(in_features=30, out_features=1, bias=True)
)


In [35]:
# 모델 사용
summary(model, input_size=(1000000,3))

Layer (type:depth-idx)                   Output Shape              Param #
IrisRegModel                             [1000000, 1]              --
├─Linear: 1-1                            [1000000, 10]             40
├─Linear: 1-2                            [1000000, 30]             330
├─Linear: 1-3                            [1000000, 1]              31
Total params: 401
Trainable params: 401
Non-trainable params: 0
Total mult-adds (M): 401
Input size (MB): 12.00
Forward/backward pass size (MB): 328.00
Params size (MB): 0.00
Estimated Total Size (MB): 340.00

[3] 데이터셋 클래스 설계 및 정의 <hr>
- 데이터셋 : iris.csv
- 피쳐 개수 : 3개
- 타겟 개수 : 1개
- 클래스 이름 : IrisDataset
- 부모 클래스 : utils.data.Dataset
- 속성 필드 : featureDF, targetDF, n_rows, n_features
- 필수 메서드
    * \__init__(self) : 데이터셋 저장 및 전처리, 개발자가 필요한 속성 설정
    * \__len__(self) : 데이터의 개수 반환
    * \__getitem__(self, index) : 특정 인덱스의 피쳐와 타겟 반환

In [36]:
class IrisDataset(Dataset):
    def __init__(self, featureDF, targetDF):
        self.featureDF = featureDF
        self.targetDF = targetDF
        self.n_rows = featureDF.shape[0]
        self.n_features = featureDF.shape[1]

    def __len__(self):
        return self.n_rows

    def __getitem__(self, index):
        # 텐서화
        featureTS = torch.FloatTensor(self.featureDF.iloc[index].values)
        targetTS = torch.FloatTensor(self.targetDF.iloc[index].values)
        
        # 피쳐와 타겟 반환
        return featureTS, targetTS

In [37]:
# 데이터셋 인스턴스 생성

# DataFrame에서 피쳐와 타겟 추출
featureDF = irisDF[irisDF.columns[:-1]] # 2D (150, 3)
targetDF = irisDF[irisDF.columns[-1:]] # 2D (150, 1)

# 커스텀 데이터셋 인스턴스
irisDS = IrisDataset(featureDF, targetDF)

[4] 학습 준비
- 학습 횟수 : EPOCH (처음부터 끝까지 학습하는 단위)
- 배치 크기 : BATCH_SIZE (한 번에 학습할 데이터셋 양)
- 위치 지정 : DEVICE (텐서 저장 및 실행 위치 : GPU/CPU) 
- 학습률 : LR 가중치와 절편 업데이트 시 경사하강법으로 업데이트 간격 설정 0.001 ~ 0.1

In [38]:
# 학습 진행 관련 설정
EPOCH = 100
BATCH_SIZE = 10
BATCH_CNT = irisDF.shape[0] // BATCH_SIZE
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
LR = 0.001

print(f'BATCH_CNT : {BATCH_CNT}')

BATCH_CNT : 15


- 인스턴스/객체 : 모델, 데이터셋, 최적화 (, 손실함수, 성능지표)

In [39]:
# 모델 인스턴스
model = IrisRegModel()

# 데이터셋 인스턴스
X_train, X_test, y_train, y_test = train_test_split(featureDF, targetDF, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, random_state=1)
print(f'{X_train.shape} {X_test.shape} {X_val.shape}')
print(f'{y_train.shape} {y_test.shape} {y_val.shape}')
print(f'{type(y_train)} {type(y_test)} {type(y_val)}')

trainDS = IrisDataset(X_train, y_train)
valDS  = IrisDataset(X_val, y_val)
testDS = IrisDataset(X_test, y_test)

# 데이터로더 인스턴스
trainDL = DataLoader(trainDS, batch_size=BATCH_SIZE)

(84, 3) (38, 3) (28, 3)
(84, 1) (38, 1) (28, 1)
<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.frame.DataFrame'> <class 'pandas.core.frame.DataFrame'>


In [40]:
# [테스트] 데이터로더
for feature, target in trainDL:
    print(feature.shape, target.shape, sep='\n')
    break

torch.Size([10, 3])
torch.Size([10, 1])


In [41]:
# 최적화 인스턴스 => W, b 텐서 즉, model.parameters() 전달
optimizer = optim.Adam(model.parameters(), lr=LR)

# 손실함수 인스턴스 => 회귀, MSE, MAE, RMSE ....
regLoss = nn.MSELoss()

[5] 학습 진행

In [42]:
# 학습의 효과 확인 손실값과 성능평가값 저장 필요
LOSS_HISTORY, SCORE_HISTORY = [[],[]], [[],[]]
CNT = irisDS.n_rows / BATCH_SIZE

# 학습 모니터링/스케쥴링 설정
# => LOSS_HISTORY, SCORE_HISTORY 활용
# => 임계기준 : 10번
BREAK_CNT_LOSS = 0
BREAK_CNT_SCORE = 0
LIMIT_VALUE = 10

for epoch in range(EPOCH):
    # 학습 모드로 모델 설정
    model.train()

    # 배치 크기 만큼 데이터 로딩해서 학습 진행
    loss_total, score_total = 0, 0

    for featureTS, targetTS in trainDL:
        # 학습 진행
        pred_y = model(featureTS)

        # 손실 계산
        loss = regLoss(pred_y, targetTS)
        loss_total += loss.item()

        # 성능평가 계산
        score = R2Score()(pred_y, targetTS)
        score_total += score.item()

        # 최적화 진행
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # 에포크 당 검증기능
    # 모델 검증 모드 설정
    model.eval()

    with torch.no_grad():
        # 검증 데이터셋
        val_featureTS = torch.FloatTensor(valDS.featureDF.values)
        val_targetTS = torch.FloatTensor(valDS.targetDF.values)

        # 추론/평가
        pred_test_y = model(val_featureTS)
        # 손실 계산
        loss_val = regLoss(pred_test_y, val_targetTS)
        # 성능 평가
        score_val = R2Score()(pred_test_y, val_targetTS)

    

    # 손실값과 성능평가값 저장
    LOSS_HISTORY[0].append(loss_total/len(trainDL))
    SCORE_HISTORY[0].append(score_total/len(trainDL))

    LOSS_HISTORY[1].append(loss_val)
    SCORE_HISTORY[1].append(score_val)

    # 학습 진행 모니터링/스케쥴링 - 검증 DS 기준
    # Loss 기준
    if len(LOSS_HISTORY[1]) >= 2:
        if LOSS_HISTORY[1][-1] >= LOSS_HISTORY[1][-2]: # 이전꺼보다 loss가 크면
            BREAK_CNT_LOSS += 1
    # Score 기준
    if len(SCORE_HISTORY[1]) >= 2:
        if SCORE_HISTORY[1][-1] <= SCORE_HISTORY[1][-2]: # 이전꺼보다 loss가 크면
            BREAK_CNT_SCORE += 1
    
    # 학습 중단 여부 설정
    if BREAK_CNT_LOSS > LIMIT_VALUE or BREAK_CNT_SCORE > LIMIT_VALUE:
        print('성능 및 손실 개선이 없어서 학습 중단')
        break

성능 및 손실 개선이 없어서 학습 중단


In [46]:
print(f"LOSS HISTORY : {LOSS_HISTORY}")
print(f"SCORE HISTORY : {SCORE_HISTORY}")

LOSS HISTORY : [[4.573483599556817, 3.4005881945292153, 2.5099964671664767, 1.8357127904891968, 1.3239113357332017, 0.9371889895863004, 0.6547954314284854, 0.4625866247547997, 0.34699640174706775, 0.28869084020455676, 0.26400438447793323, 0.2507488711012734, 0.2347881942987442, 0.20711672637197706, 0.17072881923781502, 0.13337430192364585, 0.11024918324417537, 0.09465246150890987, 0.0795706891351276, 0.0686487497554885, 0.06031610630452633, 0.05517074548535877, 0.05097135456485881, 0.04799723542398877, 0.0457283907259504, 0.04400402121245861, 0.04257621636821164, 0.04138656726313962, 0.04031812513454093, 0.039354887790977955, 0.03847916155225701, 0.037714074158834085, 0.03698910969412989, 0.03639819814513127, 0.035847209187017545, 0.035311842440730996, 0.03485633350080914, 0.0344368229723639, 0.03399569199730953, 0.03362783913811048, 0.0332681466307905, 0.032927482595874205, 0.03261301542321841, 0.032321062973803945, 0.032038364145490855, 0.03177625582449966, 0.031526008103456765, 0.03

- 모델 저장 방법 <hr>
- 방법1 : 모델 파라미터만 저장
- 방법2 : 모델 설계 구조 및 파라미터까지 모두 저장

In [47]:
# 학습된 모델 파라미터 값 확인
model.state_dict()

OrderedDict([('in_layer.weight',
              tensor([[-0.3276, -0.5475,  0.0469],
                      [-0.4278, -0.4887,  0.1337],
                      [-0.1013,  0.6490, -0.2034],
                      [ 0.5735, -0.3680, -0.4215],
                      [-0.4377,  0.1948, -0.0147],
                      [ 0.5875, -0.2994,  0.5883],
                      [-0.4828, -0.2384,  0.0803],
                      [ 0.4902,  0.3369, -0.0151],
                      [-0.5152, -0.0440,  0.1745],
                      [-0.4028,  0.2010,  0.3504]])),
             ('in_layer.bias',
              tensor([-0.3510,  0.3692, -0.4547,  0.0372, -0.3784, -0.0378, -0.5773,  0.4896,
                      -0.2251,  0.1789])),
             ('hidden_layer.weight',
              tensor([[ 3.5554e-02, -8.6836e-03, -1.6969e-02, -3.1039e-01, -2.0200e-01,
                       -2.7776e-01, -1.9533e-01,  2.5943e-02,  2.0766e-01,  3.5518e-04],
                      [ 1.7280e-01,  2.8542e-03, -3.1032e-02,  3.0955e-0

- [방법 1] 모델 파라미터 즉, 층별 가중치와 절편들

In [52]:
# models 폴더 아래 프로젝트 폴더 아래 모델 파일 저장
import os 

# 저장 경로
SAVE_PATH = 'models/iris/'
# 저장 파일명
SAVE_FILE = 'model_train_wbs.pth'

In [54]:
# 경로상 폴더 존재 여부 체크
if not os.path.exists(SAVE_PATH):
    os.makedirs(SAVE_PATH) # 폴더 생성

In [56]:
# 모델 저장
torch.save(model.state_dict(), SAVE_PATH+SAVE_FILE)

In [59]:
# 모델 즉, 가중치와 절편 로딩
# [1] 가중치와 절편을 객체로 로딩
# [2] 모델의 state_dict 속성에 저장

# 읽기
wbTS=torch.load(SAVE_PATH+SAVE_FILE, weights_only=True)
print(type(wbTS))

<class 'collections.OrderedDict'>


In [61]:
# 모델 인스턴스에 저장
model2 = IrisRegModel() # 층마다 W, b 초기화
model2.load_state_dict(wbTS)

<All keys matched successfully>

In [62]:
print(model2)

IrisRegModel(
  (in_layer): Linear(in_features=3, out_features=10, bias=True)
  (hidden_layer): Linear(in_features=10, out_features=30, bias=True)
  (out_layer): Linear(in_features=30, out_features=1, bias=True)
)
